In [498]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import tree
import os
import torch 
from decimal import Decimal
from torch.utils.data import Dataset, DataLoader,TensorDataset
import numpy as np
import math
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import matplotlib.pyplot as plt
import multiprocessing
import glob
import os
from scipy import linalg
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from torch.autograd import Variable 

In [499]:
df=pd.read_csv('data_with_ioa.csv')

In [500]:
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [501]:
df['label'].value_counts()


0    2235488
2       6343
1       2289
4       1344
5       1270
6        964
3        499
Name: label, dtype: int64

In [502]:
#majority under sampling
# df
# data=df[df['label']!=0]
# data_s = resample(df[df['label']==0], replace=False,n_samples=10000,random_state=123)
# df=pd.concat([data,data_s])

In [503]:
#All sampling
data_train_sampled=pd.DataFrame([],columns=df.columns.values,dtype=float)
sampling=2000
for i in df['label'].unique():
    if df[df['label']==i].shape[0]>sampling:
        
        data_s = resample(df[df['label']==i], replace=False,n_samples=sampling,random_state=123)
        data_train_sampled=pd.concat([data_train_sampled,data_s])
    else:
        data_s = resample(df[df['label']==i], replace=True,n_samples=sampling,random_state=123) 
        data_train_sampled=pd.concat([data_train_sampled,data_s])

In [504]:
# X,y = df.iloc[:, :-1], df.iloc[:, [-1]]
X, y = data_train_sampled.iloc[:, :-1], data_train_sampled.iloc[:, [-1]]



In [505]:
X

,TimeStamp,Relative Time,srcIP,dstIP,srcPort,dstPort,ipLen,len,fmt,uType,asduType,numix,cot,oa,addr,ioa
889949,18.081897,179644.840967,1.677724e+08,1.677725e+08,2404.0,49716.0,58.0,16.0,1.0,2.0,9.0,1.0,3.0,0.0,55.0,136.0
153357,9.370158,154524.584637,3.232239e+09,3.232238e+09,2404.0,61254.0,67.0,25.0,1.0,2.0,36.0,1.0,3.0,0.0,65535.0,1.0
510812,2.384546,37471.326137,1.677724e+08,1.677725e+08,2404.0,49716.0,58.0,16.0,1.0,2.0,9.0,1.0,3.0,0.0,55.0,136.0
65642,21.315824,24621.243906,3.232239e+09,3.232238e+09,2404.0,61254.0,67.0,25.0,1.0,2.0,36.0,1.0,3.0,0.0,65535.0,1.0
108074,0.434399,208926.992331,3.232239e+09,3.232238e+09,2404.0,61254.0,67.0,25.0,1.0,2.0,36.0,1.0,3.0,0.0,65535.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400035,21.175666,196579.662383,3.232239e+09,3.232238e+09,2404.0,61254.0,67.0,25.0,1.0,2.0,36.0,1.0,3.0,0.0,65535.0,2.0
399869,21.064536,195908.361039,3.232238e+09,3.232239e+09,61254.0,2404.0,46.0,4.0,2.0,2.0,9.0,1.0,3.0,0.0,55.0,136.0
359038,21.163942,23702.428604,3.232239e+09,3.232238e+09,2404.0,61254.0,46.0,4.0,0.0,2.0,9.0,1.0,3.0,0.0,55.0,136.0
358923,21.082679,23209.794125,3.232238e+09,3.232239e+09,61254.0,2404.0,46.0,4.0,2.0,2.0,9.0,1.0,3.0,0.0,55.0,136.0


In [506]:
y['label'].value_counts()


0.0    2000
1.0    2000
2.0    2000
3.0    2000
4.0    2000
5.0    2000
6.0    2000
Name: label, dtype: int64

In [507]:
mm = MinMaxScaler()

X_mm = mm.fit_transform(X) 


df_data=torch.Tensor(X_mm)
df_label=y.values[:,0]

# y=torch.Tensor(y.values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = df_label.reshape(len(df_label), 1)
encoded_label = onehot_encoder.fit_transform(integer_encoded)
encoded_label=torch.Tensor(encoded_label)




In [508]:
X_train, X_test, y_train, y_test = train_test_split(df_data, encoded_label, test_size=0.3,random_state=109) # 70% training and 30% test


In [509]:
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, shuffle=True, batch_size=train_data.tensors[0].shape[0])
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0])

print("Training data batches:")
for X, y in train_loader:
    print(X.shape, y.shape)
    
print("\nTest data batches:")
for X, y in test_loader:
    print(X.shape, y.shape)

Training data batches:
torch.Size([9800, 16]) torch.Size([9800, 7])

Test data batches:
torch.Size([4200, 16]) torch.Size([4200, 7])


In [510]:

#Option 1
in_f=train_data.tensors[0].shape[1]
out_f=train_data.tensors[1].shape[1]

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(in_features=in_f, out_features=128)
        self.hidden_1 = nn.Linear(in_features=128, out_features=64)
        self.hidden_12 = nn.Linear(in_features=64, out_features=32)

        self.output = nn.Linear(in_features=32, out_features=out_f)
        
        
        
    def forward(self, x):
        x = F.sigmoid(self.input(x))
        x = F.sigmoid(self.hidden_1(x))
        x = F.sigmoid(self.hidden_12(x))

    
        return self.output(x)
    
    
model = Net()
print(model)

Net(
  (input): Linear(in_features=16, out_features=128, bias=True)
  (hidden_1): Linear(in_features=128, out_features=64, bias=True)
  (hidden_12): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=7, bias=True)
)


In [511]:
#Option 2

class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [512]:
model = MulticlassClassification(num_feature = in_f, num_class=out_f)

In [513]:
num_epochs = 300

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.005)

for epoch in range(num_epochs):
    for X, y in train_loader:
        preds = model(X)
        preds=torch.softmax(preds,axis=1)
        
        loss = loss_function(preds, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        y_labels=torch.argmax(y,axis=1)
        pred_labels=torch.argmax(preds,axis=1)
        
        acc=100 * torch.mean((pred_labels == y_labels).float()).item()
            
    print(metrics.classification_report(y_labels, pred_labels))

    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss.item(), acc))
    X , y = next(iter(test_loader))
    preds=torch.softmax(model(X),axis=1)
    y_labels=torch.argmax(y,axis=1)
    pred_labels=torch.argmax(preds,axis=1)
    testacc=100 * torch.mean((pred_labels == y_labels).float()).item()
    print('Test Accuracy:', testacc)


              precision    recall  f1-score   support

           0       0.20      0.08      0.12      1394
           1       0.13      0.25      0.17      1378
           2       0.11      0.07      0.08      1410
           3       0.17      0.42      0.25      1396
           4       0.14      0.11      0.12      1377
           5       0.09      0.03      0.05      1440
           6       0.23      0.11      0.15      1405

    accuracy                           0.15      9800
   macro avg       0.15      0.15      0.13      9800
weighted avg       0.15      0.15      0.13      9800

Epoch 1/400, Loss: 1.951, Accuracy: 15.306
Test Accuracy: 41.5714293718338
              precision    recall  f1-score   support

           0       0.94      0.79      0.86      1394
           1       0.20      0.49      0.28      1378
           2       0.32      0.10      0.15      1410
           3       0.26      0.37      0.31      1396
           4       0.44      0.40      0.42      1377
   

              precision    recall  f1-score   support

           0       0.99      0.81      0.89      1394
           1       0.89      0.17      0.28      1378
           2       0.43      0.70      0.53      1410
           3       0.44      0.43      0.44      1396
           4       0.63      0.54      0.58      1377
           5       0.65      0.67      0.66      1440
           6       0.67      0.99      0.80      1405

    accuracy                           0.62      9800
   macro avg       0.67      0.62      0.60      9800
weighted avg       0.67      0.62      0.60      9800

Epoch 14/400, Loss: 1.580, Accuracy: 61.724
Test Accuracy: 63.11904788017273
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.84      0.20      0.32      1378
           2       0.47      0.71      0.57      1410
           3       0.49      0.53      0.51      1396
           4       0.65      0.54      0.59      1377
 

              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.80      0.67      0.73      1378
           2       0.72      0.87      0.79      1410
           3       0.83      0.97      0.90      1396
           4       0.85      0.44      0.58      1377
           5       0.75      0.99      0.85      1440
           6       0.90      1.00      0.94      1405

    accuracy                           0.82      9800
   macro avg       0.84      0.82      0.81      9800
weighted avg       0.84      0.82      0.81      9800

Epoch 27/400, Loss: 1.412, Accuracy: 82.337
Test Accuracy: 82.35714435577393
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.81      0.72      0.76      1378
           2       0.71      0.87      0.78      1410
           3       0.86      0.97      0.91      1396
           4       0.86      0.42      0.57      1377
 

              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.80      0.84      0.82      1378
           2       0.73      0.91      0.81      1410
           3       0.97      0.99      0.98      1396
           4       0.94      0.44      0.60      1377
           5       0.79      1.00      0.88      1440
           6       0.90      1.00      0.95      1405

    accuracy                           0.86      9800
   macro avg       0.87      0.86      0.85      9800
weighted avg       0.87      0.86      0.85      9800

Epoch 41/400, Loss: 1.329, Accuracy: 85.765
Test Accuracy: 85.50000190734863
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.80      0.84      0.82      1378
           2       0.74      0.91      0.82      1410
           3       0.95      0.99      0.97      1396
           4       0.94      0.45      0.60      1377
 

Test Accuracy: 86.6190493106842
              precision    recall  f1-score   support

           0       1.00      0.80      0.89      1394
           1       0.79      0.85      0.82      1378
           2       0.75      0.94      0.84      1410
           3       0.97      1.00      0.98      1396
           4       0.98      0.47      0.63      1377
           5       0.81      1.00      0.89      1440
           6       0.90      1.00      0.95      1405

    accuracy                           0.87      9800
   macro avg       0.88      0.86      0.86      9800
weighted avg       0.88      0.87      0.86      9800

Epoch 56/400, Loss: 1.306, Accuracy: 86.643
Test Accuracy: 86.47618889808655
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.79      0.85      0.82      1378
           2       0.75      0.94      0.83      1410
           3       0.97      1.00      0.98      1396
           4       0.98

...

Test Accuracy: 88.83333206176758
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.83      1.00      0.91      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.99      0.48      0.64      1377
           5       0.89      1.00      0.94      1440
           6       0.90      1.00      0.95      1405

    accuracy                           0.89      9800
   macro avg       0.91      0.89      0.88      9800
weighted avg       0.91      0.89      0.88      9800

Epoch 332/400, Loss: 1.276, Accuracy: 89.041
Test Accuracy: 88.88095021247864
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.85      0.99      0.92      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.

Test Accuracy: 88.88095021247864
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.85      1.00      0.92      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.99      0.48      0.64      1377
           5       0.87      1.00      0.93      1440
           6       0.90      1.00      0.95      1405

    accuracy                           0.89      9800
   macro avg       0.91      0.89      0.88      9800
weighted avg       0.91      0.89      0.88      9800

Epoch 346/400, Loss: 1.275, Accuracy: 89.031
Test Accuracy: 88.9047622680664
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.85      1.00      0.92      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.9

Test Accuracy: 89.04761672019958
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.85      1.00      0.92      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.99      0.48      0.64      1377
           5       0.87      1.00      0.93      1440
           6       0.90      1.00      0.95      1405

    accuracy                           0.89      9800
   macro avg       0.91      0.89      0.88      9800
weighted avg       0.91      0.89      0.88      9800

Epoch 360/400, Loss: 1.275, Accuracy: 89.041
Test Accuracy: 88.97619247436523
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.85      1.00      0.92      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.

Test Accuracy: 88.95238041877747
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.85      1.00      0.92      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.99      0.47      0.64      1377
           5       0.87      1.00      0.93      1440
           6       0.90      1.00      0.95      1405

    accuracy                           0.89      9800
   macro avg       0.91      0.89      0.88      9800
weighted avg       0.91      0.89      0.88      9800

Epoch 374/400, Loss: 1.275, Accuracy: 89.020
Test Accuracy: 88.97619247436523
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.87      1.00      0.93      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.

Test Accuracy: 89.04761672019958
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.85      1.00      0.92      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.99      0.48      0.64      1377
           5       0.87      1.00      0.93      1440
           6       0.90      1.00      0.95      1405

    accuracy                           0.89      9800
   macro avg       0.91      0.89      0.88      9800
weighted avg       0.91      0.89      0.88      9800

Epoch 388/400, Loss: 1.275, Accuracy: 89.051
Test Accuracy: 88.99999856948853
              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1394
           1       0.84      1.00      0.91      1378
           2       0.76      0.94      0.84      1410
           3       0.98      1.00      0.99      1396
           4       0.